In [1]:
import glob as glob
import cv2 as cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
images = glob.glob('./SodaBottles/M.Beer/*.jpg')
print(images[0:5])

['./SodaBottles/M.Beer\\1351_corrected.jpg', './SodaBottles/M.Beer\\1425_corrected.jpg', './SodaBottles/M.Beer\\1438_corrected.jpg', './SodaBottles/M.Beer\\1581_corrected.jpg', './SodaBottles/M.Beer\\1605_corrected.jpg']


In [4]:
def change_light(image, coeff):
    image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS) ## Conversion to HLS
    image_HLS = np.array(image_HLS, dtype = np.float64) 
    image_HLS[:,:,1] = image_HLS[:,:,1]*coeff ## scale pixel values up or down for channel 1(Lightness)
    if(coeff>1):
        image_HLS[:,:,1][image_HLS[:,:,1]>255]  = 255 ##Sets all values above 255 to 255
    else:
        image_HLS[:,:,1][image_HLS[:,:,1]<0]=0
    image_HLS = np.array(image_HLS, dtype = np.uint8)
    image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB) ## Conversion to RGB
    return image_RGB 


In [ ]:
for i in images:
    print(i.split('\\')[-1])
    
    filename = i.split('\\')[-1]
    
    modified_path = i.replace("M.Beer", "Cropped_M.Beer").split('\\')[0]
    
    local_image = cv2.imread(i, cv2.COLOR_BGR2RGB)
    
    crop_img = local_image[100:380, 100:]
    
    
    cv2.imwrite(modified_path +"/"+ filename,crop_img)
    
    
    #cv2.imwrite(modified_path +"/BR_"+ filename,bright_image)
    
    flip_right_img=cv2.flip(crop_img,1)
    cv2.imwrite(modified_path +"/RF_"+ filename,flip_right_img)
    
    flip_left_img = cv2.flip(crop_img,0)
    cv2.imwrite(modified_path +"/LF_"+ filename,flip_left_img)
    
    crop_img = change_light(crop_img, 2)
    flip_right_img = change_light(flip_right_img, 2)
    flip_left_img = change_light(flip_left_img, 2)
    
    (h, w) = crop_img.shape[:2]
    #print("image dimensions are ",w,h)
    center = (w / 2, h / 2)

    angles=[10,20,30,40,50,60]
    scale = 1.0

    # Perform the counter clockwise rotation holding at the center
    # 90 degrees
    for i in angles:
        M = cv2.getRotationMatrix2D(center, i, scale)
        rotated90 = cv2.warpAffine(crop_img, M, (w, h))
        cv2.imwrite(modified_path +"/"+str(i)+"_"+ filename,rotated90)
        
        M = cv2.getRotationMatrix2D(center, i, scale)
        rotated90 = cv2.warpAffine(flip_right_img, M, (w, h))
        cv2.imwrite(modified_path +"/"+str(i)+"RF_"+ filename,rotated90)
        
        M = cv2.getRotationMatrix2D(center, i, scale)
        rotated90 = cv2.warpAffine(flip_left_img, M, (w, h))
        cv2.imwrite(modified_path +"/"+str(i)+"LF_"+ filename,rotated90)
     
    
    #plt.show()
print("Done!")

In [3]:
images_new = []
for i in images:
    images_new.append(cv2.imread(i))
new_X = np.array(images)
print(new_X.shape)


(840,)


In [4]:
new_X = np.array(images_new)
print(new_X.shape)
#new_X = new_X.reshape(new_X.shape[0],3,640,480)
#print(new_X.shape)

(840, 480, 640, 3)


In [5]:
X_train = new_X.astype('float32')



In [1]:
import tensorflow as tf

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

datagen.fit(X_train)

Using TensorFlow backend.


In [11]:
datagen.flow(X_train, batch_size=9, save_to_dir='./SodaBottles/New_Cropped_M.Beer', save_prefix='aug', save_format='jpg')

c:\users\kanwal\miniconda3\lib\site-packages\keras_preprocessing\image.py:1404: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (840, 3, 640, 480) (480 channels).
  str(self.x.shape[channels_axis]) + ' channels).')
